# Transforming Text into Features Using Word Embeddings

### Import Packages

In [1]:
import pandas as pd
import numpy as np
import os 
import matplotlib as plt
import seaborn as sns

We will also import the scikit-learn `LogisticRegression class`, the `train_test_split()` function for splitting the data into training and test sets, and the `roc_auc_score()` function to evaluate the model.


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

## Step 1: Build DataFrame and Define ML Problem

We will work with a dataset containing emails that are labeled as either spam or not spam.

#### Load a Data Set and Save it as a Pandas DataFrame

In [3]:
filename = os.path.join(os.getcwd(), "..", "data", "spamDataset.csv")
df = pd.read_csv(filename, header=0)

df.head()

,email_text,spam
0,Subject: enron methanol ; meter # : 988291\r\n...,False
1,"Subject: hpl nom for january 9 , 2001\r\n( see...",False
2,"Subject: neon retreat\r\nho ho ho , we ' re ar...",False
3,"Subject: photoshop , windows , office . cheap ...",True
4,Subject: re : indian springs\r\nthis deal is t...,False


#### Define the Label

This is a binary classification problem in which we will predict whether an email is a spam or not spam. The label is the `spam` column.

#### Identify Features

We only have one feature. The feature is the `email_text` column.


## Step 2: Create Labeled Examples from the Data Set

Let's create labeled examples from our dataset. We will have one text feature and one label. 
The code cell below carries out the following steps:

* Gets the `spam` column from DataFrame `df` and assigns it to the variable `y`. This will be our label. Note that the label contains True or False values that indicate whether an email is spam or not spam.
* Gets the column `email_text` from DataFrame `df` and assigns it to the variable `X`. This will our feature. Note that the `email_text` feature contains the subject line of the email.


In [4]:
y = df['spam'] 
X = df['email_text']

X.shape

(5170,)

In [5]:
X.head()

0    Subject: enron methanol ; meter # : 988291\r\n...
1    Subject: hpl nom for january 9 , 2001\r\n( see...
2    Subject: neon retreat\r\nho ho ho , we ' re ar...
3    Subject: photoshop , windows , office . cheap ...
4    Subject: re : indian springs\r\nthis deal is t...
Name: email_text, dtype: object

Let's take a look at an example of a spam email and an email that is not spam.

In [6]:
print('A spam email: \n\n', X[67])
print('A non-spam email: \n\n', X[135])

A spam email: 

 Subject: re : husband soup would be
as you know election time is not the best thing for the economy .
economy is in a very unstable condition , as you can see gas prices
are going up along with the m o rtgvage rat e s . once the
r a te goes up you will not have a chance to s av e money again
for a very long time .
it is your last chance . get r e f inanced at 4 . 2 point !
http : / / www . fintod . com /
- -
despoil , compote a amende
the me orbital irruption
gfawn a ax henrietta
a the in boatswain
out whither the accompanist lint macintosh

A non-spam email: 

 Subject: re : tuesday , december 26 th
i will be here tuesday , also .
mark mccoy
12 / 20 / 2000 09 : 04 am
to : michael olsen / na / enron @ enron , tom acton / corp / enron @ enron , clem
cernosek / hou / ect @ ect , robert cotten / hou / ect @ ect , jackie young / hou / ect @ ect ,
sabrae zajac / hou / ect @ ect , carlos j rodriguez / hou / ect @ ect , mark
mccoy / corp / enron @ enron , aimee lannou / hou /

## Step 3: Preprocess the Text

The next step is to preprocess the text. Preprocessing technqiues can include cleaning the data, converting all text to lowercase, removing special characters, removing stopwords from the text, tokenizing the text (split it into smaller chunks), and lemmatizing the text (converting a word to its root word).

You can perform preprocessing on your own or use tools to accomplish this. One common tool is NLTK [Natural Language Toolkit](https://www.nltk.org/). However, for this demo, we will use the built-in function from Gensim to preprocess the text. This function will remove some stop words, covert all text to lowercase, remove punctuation and tokenize the text.

Let's import the Gensim package.

In [7]:
import gensim

Before we preprocess our data, let's look at an example output of preprocessed text. Let's take a simple sentence and perform preprocessing. Run the code cell below and inspect the results.

In [8]:
sentence = "I went to the market to buy some apples for my pet horse." 

list(gensim.utils.simple_preprocess(sentence))

['went',
 'to',
 'the',
 'market',
 'to',
 'buy',
 'some',
 'apples',
 'for',
 'my',
 'pet',
 'horse']

Now let's perform preprocessing on our `email_text` feature and compare the difference between the original text and the preprocessed text

In [9]:
original_X = X
X = X.apply(lambda row: gensim.utils.simple_preprocess(row))

In [10]:
X.head()

0    [subject, enron, methanol, meter, this, is, fo...
1    [subject, hpl, nom, for, january, see, attache...
2    [subject, neon, retreat, ho, ho, ho, we, re, a...
3    [subject, photoshop, windows, office, cheap, m...
4    [subject, re, indian, springs, this, deal, is,...
Name: email_text, dtype: object

In [11]:
original_X.head()

0    Subject: enron methanol ; meter # : 988291\r\n...
1    Subject: hpl nom for january 9 , 2001\r\n( see...
2    Subject: neon retreat\r\nho ho ho , we ' re ar...
3    Subject: photoshop , windows , office . cheap ...
4    Subject: re : indian springs\r\nthis deal is t...
Name: email_text, dtype: object

## Step 4: Create Training and Test Data Sets

Let's take our preprocessed text data set and split the data into training and test sets with 80% of the data being the training set.

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.80, random_state=1234)

X_train.head()

562     [subject, hpl, nom, for, sept, see, attached, ...
148     [subject, copanno, changes, forwarded, by, ami...
4831    [subject, re, maynard, oil, revised, nom, dare...
3385    [subject, deal, for, december, can, either, of...
2389    [subject, heisse, sx, action, hallo, mein, lie...
Name: email_text, dtype: object

## Step 5: Training the Word2Vec Model and Inspecting the Word Embeddings

Now that the data has been preprocessed and we have our training data, we will train a Word2Vec model using Gensim using the training data `X_train`. For more information on Gensim, consult the online [documentation](https://radimrehurek.com/gensim/models/word2vec.html). The model will produce word embeddings. That is, words represented as numerical vectors based on how these words are used in the email spam dataset.
    
We will specify the following parameters:

* size: dimension of each resulting vector that will represent each word.
* window: the number of words behind or ahead of a target word that will be used to provide context for that word
* min_count: the number of times a word must appear in our text document in order to create a word vector. The model will ignore words that do not satisfy the `min_count` specification, therefore ignoring words that are not important.



In [13]:
print("Begin")
word2vec_model = gensim.models.Word2Vec(X_train,
                                   vector_size=100,
                                   window=5,
                                   min_count=2)

print("End")

Begin
End


You can check the size of a particular model with the `len()` method. 

In [14]:
len(word2vec_model.wv.key_to_index)  # retrieve vocabulary and measure its size

18015

While 18,015 words may seem like a lot, this is actually quite small for most real-world applications. In practice, you can use models with larger vector sizes (say 300) and much larger vocabulary.

To check if a word is included the vocabulary, you can use the `key_to_index` attribute. 

In [15]:
'cornell' in word2vec_model.wv.key_to_index

False

In [16]:
'dog' in word2vec_model.wv.key_to_index

True

The word `'dog'` is in our vocabulary. Let's inspect its word vector. The code cell belows retrieves the vector for the word `'dog'`. 

In [17]:
word2vec_model.wv['dog']

array([-0.04492886,  0.21974644,  0.03547766, -0.23517995,  0.02904211,
       -0.16802655,  0.07486057,  0.41911623, -0.23018387, -0.17165902,
       -0.23773709, -0.31308135, -0.00583143,  0.00578949,  0.18014345,
       -0.29475018,  0.07711259, -0.21995772, -0.01681948, -0.35177317,
        0.14860125,  0.10346006,  0.19733422, -0.17530908,  0.00630486,
        0.08640727, -0.22077747, -0.17459418, -0.21438915, -0.02494479,
        0.19054097, -0.03065328,  0.04631107, -0.33309138, -0.1254988 ,
        0.13406509, -0.068528  , -0.09115726, -0.15578236, -0.37402678,
        0.01954006, -0.0343144 , -0.09579159,  0.08276404,  0.18221949,
       -0.1103784 , -0.05531504, -0.14377621,  0.10669465,  0.23473403,
        0.02276917, -0.09967692, -0.2627176 , -0.12282371, -0.20631208,
        0.00384213, -0.02264421,  0.02399508, -0.20974542,  0.03701084,
        0.06527066,  0.0484629 ,  0.09466923, -0.01640165, -0.2103403 ,
        0.10407037,  0.25413656,  0.12144031, -0.24319811,  0.16

Examine the vector, noting each of the following important characteristics: 

1. It contains 100 values somewhere between -0.39333 and 0.4495
2. There are no zeros, so it can be considered a **dense vector** (a vector comprised of mostly non-zero values)
3. Each value represents a dimension, but is not necessarily interpretable by humans
4. Larger (in magnitude) values may relate to categories in which `'dog'` has a presence
5. Many broader categories are also likely to be represented by a combination of these dimensions
6. A few values are relatively close to zero. This implies that these dimensions do not carry "significant information" about `'dog'`


Let's find the most similar words for `'dog'`.


In [18]:
word2vec_model.wv.most_similar('dog')

[('magazine', 0.9915481805801392),
 ('artprice', 0.989270031452179),
 ('bereave', 0.9889470338821411),
 ('mental', 0.9887989163398743),
 ('baseplate', 0.9880924224853516),
 ('barrage', 0.9880189895629883),
 ('hospitable', 0.9879831075668335),
 ('author', 0.9879058003425598),
 ('wy', 0.9877417087554932),
 ('pour', 0.987568736076355)]

The code cell below outputs the first few 25 words that our Word2Vec model learned a vector for.

In [19]:
top25 = word2vec_model.wv.index_to_key[:25]
top25

['the',
 'to',
 'ect',
 'and',
 'for',
 'of',
 'you',
 'subject',
 'in',
 'on',
 'is',
 'this',
 'hou',
 'enron',
 'be',
 'that',
 'we',
 'from',
 'your',
 'will',
 'have',
 'with',
 'at',
 'are',
 'it']

You can go a step further and print vectors associated with each of these words. Use the `background_gradient` method to wrap these vectors into a DataFrame as rows with the corresponding words as row indices. Let's spice it up with colors.

In [20]:
pd.DataFrame({w:word2vec_model.wv[w] for w in top25}).T.style.background_gradient(cmap='coolwarm')

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
the,-0.998629,1.506672,-0.375423,0.318331,-0.788028,-2.478503,1.078627,1.362659,0.154688,-1.067723,0.868921,-0.629299,-0.379828,1.228719,-0.017436,1.179010,-0.608044,-0.975191,0.405029,-0.909487,1.541962,0.221362,1.687882,0.052167,-1.089603,-0.162081,0.576203,0.813391,-0.081897,-0.848427,-0.007786,-0.522995,0.401410,-0.591358,1.037212,0.427046,1.116599,-0.412807,-0.032047,-0.111769,-1.266242,-1.014240,-1.254112,0.578914,1.092400,0.208895,-1.575518,1.212084,1.862612,0.991250,-0.233654,0.784937,0.375876,1.358220,1.224973,0.504106,1.433008,0.105345,-1.031084,1.199660,-0.955816,1.336862,-1.248032,-0.776492,0.305849,-0.297833,-0.194773,1.352495,0.283985,0.717622,-0.068143,0.047591,-0.408576,0.806121,0.432514,-0.256899,-0.266762,-0.157374,-0.167693,-1.236087,-0.392356,1.025615,0.944677,-0.028762,1.399611,0.165356,-0.007727,0.762969,0.814220,1.853768,1.684970,-0.450972,-1.245333,0.261677,1.941906,1.741329,-0.694396,-1.157756,-0.185791,0.532247
to,-1.593720,1.444328,-0.211883,1.527166,-0.362020,-1.230098,1.845853,-0.068213,0.620426,-0.134682,1.047291,-0.085672,-0.791580,0.449867,-0.468715,0.608018,1.426049,-0.158058,0.176682,-1.087154,1.357685,0.147804,-0.354094,-1.123152,0.352428,0.025622,-0.870739,0.465962,-1.023573,0.692372,1.332838,0.334848,0.998699,-0.219209,0.133655,2.088745,0.808223,-0.090804,-0.278592,-0.523619,1.458068,-1.653486,-1.071369,0.444177,-0.152662,1.553810,0.069545,-0.306577,1.460077,-0.714229,0.947643,-1.249801,0.199308,1.782188,-0.494940,0.622424,1.141650,-0.430317,-0.647312,0.566794,-0.391360,0.797014,-0.073842,-0.542127,0.189924,1.329361,-0.607223,0.569974,-0.847725,-0.529554,0.135678,-0.202694,0.991505,1.381598,1.302761,-1.994463,0.333066,-0.107269,1.113328,-0.554189,-0.622894,0.344758,0.327871,-0.575861,1.239138,-0.437840,0.014920,-0.915921,0.747345,1.133098,2.126022,1.145731,-0.505972,-0.712098,-0.287198,0.224771,-0.838133,0.876354,0.028524,0.501961
ect,-0.491349,1.055445,-1.596955,0.762279,-0.562767,1.904329,0.223696,2.725984,-1.000918,-1.024644,-0.756402,-0.743480,-0.909507,-1.664040,0.247794,-1.999983,1.457215,3.678990,-0.201492,-1.189899,-2.715105,0.073828,0.546228,-2.068287,0.694721,1.607139,-0.004615,1.867726,-3.009612,1.988567,-0.474555,0.060041,0.258017,0.141890,-1.085060,-0.653354,-1.993185,0.592890,-0.793958,-0.601823,-1.326111,-0.158451,-2.221634,0.771982,3.569505,-3.109259,1.060175,-3.319534,-0.058979,1.050813,0.125194,-1.082846,-0.254404,-1.446671,-1.467690,-1.417921,-0.054484,-2.623811,-1.475177,-2.319983,0.525854,-0.897266,2.872788,-0.291893,-3.244733,3.592330,0.617095,2.228452,-6.883261,1.539843,-0.335313,4.118303,3.138077,0.748716,1.467274,3.127907,2.815874,-0.122143,0.264215,-0.941655,-2.011672,-1.439967,-1.011153,0.186234,-0.446700,-2.195197,-0.497472,-2.832766,1.499328,2.168817,1.154398,0.648929,2.158712,-0.330468,4.548630,1.488480,1.790416,1.677758,0.128134,-2.551702
and,-0.554302,1.186027,-0.217192,0.372219,-0.201710,-1.052285,0.845762,1.047877,-0.110340,-0.723533,0.657357,-0.665297,-0.545807,0.124374,0.463268,0.583129,0.419058,-0.793821,-0.096030,-0.720874,0.708627,0.113516,1.129378,0.001749,-0.757639,0.281303,-0.128707,0.050674,0.302654,0.044114,0.476162,-0.056905,-0.064894,-0.769022,0.577230,1.252480,1.004129,-0.345583,-0.325249,-0.291516,0.472656,-1.243195,-0.803752,0.492605,0.777289,0.742617,-1.086379,0.136658,1.118337,0.295662,0.673765,-0.058391,-0.041097,0.572599,-0.515840,0.406976,1.117964,0.310886,-0.321028,0.198478,0.047718,0.482791,-0.480013,-0.233334,0.189037,0.520648,-0.149052,0.462650,-0.103901,0.377849,-0.033398,-0.714381,-0.013194,0.636729,0.404324,-0.426529,-0.559491,-0.067732,0.364777,-0.494573,0.011516,0.261435,0.511590,-0.399735,0.508914,0.032104,0.079215,0.670980,0.402863,0.519734,

Note that the 100 columns you see here are dimensions you specified earlier. We do not know what each dimension signifies, although this is an active area of research. However, if you examine each dimension you'll notice more red in some columns and more blue in others. Some of the words are very common and might be stop words we would normally remove. However, if we load more specific groups of words, we might make good guesses about what some of the dimensions mean.

The code cells below compute the similarities between two words in the vocabulary.

In [21]:
word2vec_model.wv.similarity(w1='dog', w2='person')

0.3747899

In [22]:
word2vec_model.wv.similarity(w1='dog', w2='cat')

0.96203583

## Step 6: Create Feature Vectors out of Word Embeddings for a Classifier

Now let's convert the features in our training and test datasets into feature vectors using our word embeddings. We will use this  to train a logistic regression model. Let's first inspect our original training and test datasets:

In [23]:
X_train.head()

562     [subject, hpl, nom, for, sept, see, attached, ...
148     [subject, copanno, changes, forwarded, by, ami...
4831    [subject, re, maynard, oil, revised, nom, dare...
3385    [subject, deal, for, december, can, either, of...
2389    [subject, heisse, sx, action, hallo, mein, lie...
Name: email_text, dtype: object

In [24]:
X_test.head()

5075    [subject, hpl, nom, for, may, see, attached, f...
3817    [subject, cleburne, tenaska, iv, plant, daren,...
2967    [subject, re, producer, connects, on, the, con...
3014    [subject, mail, hey, daren, is, this, your, ma...
2185    [subject, revision, forest, oil, november, gas...
Name: email_text, dtype: object

The code cell below walks through every example in both the training and test datasets and replaces every word contained in the `email_text` feature with its corresponding word embedding. Original words that do not have a corresponding word embedding will not be part of the training and test sets. For example, stop words that were removed when creating the word embeddings will not appear in the training and test sets.

<b>Note</b>: This may take a while to run.

In [25]:
words = set(word2vec_model.wv.index_to_key)

print('Begin transforming X_train')
X_train_word_embeddings = np.array([np.array([word2vec_model.wv[word] for word in words if word in training_example])
                        for training_example in X_train], dtype=object)
print('Finish transforming X_train')

print('Begin transforming X_test')
X_test_word_embeddings = np.array([np.array([word2vec_model.wv[word] for word in words if word in training_example])
                        for training_example in X_test], dtype=object)
print('Finish transforming X_test')


Begin transforming X_train
Finish transforming X_train
Begin transforming X_test
Finish transforming X_test


In [26]:
print('Number of words in first training example: {0}'.format(len(X_train.iloc[0])))
print('First word in first training example: {0}'.format(X_train.iloc[0][0]))
print('Second word in first training example: {0}\n'.format(X_train.iloc[0][1]))

print('First word vector in first training example:\n {0}'.format(X_train_word_embeddings[0][0]))
print('\nSecond word vector in first training example: \n {0}\n'.format(X_train_word_embeddings[0][1]))


Number of words in first training example: 12
First word in first training example: subject
Second word in first training example: hpl

First word vector in first training example:
 [-2.6507356  -0.54793656 -0.6966936  -2.0907614   0.70093405  0.37078598
 -0.07101369  2.463107   -0.41962403 -0.69993925 -2.0295115  -1.5894176
  0.5589172  -2.0024247   0.06885199 -2.0697203  -0.26318556 -0.15277642
  1.4368387  -1.2089779   3.414807    2.1660573  -0.2756688  -0.639335
  0.4353633   1.4229956  -3.544587   -1.5467875  -2.1409128   0.23103064
 -2.6091132   2.402737    0.07015277 -1.4544841   1.7471577   1.2870162
 -1.2990102  -0.13854921  0.1341196  -2.3038356   0.03491662  2.1348922
 -2.324907    1.2680665   2.505025   -3.0530486  -0.2853834  -1.3885959
  0.8292451   1.1982154   0.4225875  -0.40855956 -1.4715606  -0.8287263
 -0.08915003  1.3812134   1.8580225   0.7923013  -1.4435124   0.76383215
 -0.7060326   0.48420468  0.6821564  -0.857825   -3.5747147  -0.44823006
  1.5579196   0.367120

After replacing the `email_text` feature in our training and test data with word embeddings, each example in our training and test data now has a different number of features, each corresponding to a word vector:

In [27]:
print('Number of word vectors in first five examples in training set:')
for w in range(0, 5):
    print(len(X_train_word_embeddings[w]))

print('Number of word vectors in first five examples in test set:')
for w in range(0, 5):
    print(len(X_test_word_embeddings[w]))

Number of word vectors in first five examples in training set:
10
23
37
57
30
Number of word vectors in first five examples in test set:
10
31
59
8
59


This will cause an error when we train our model. We have to create feature vectors that will provide our classifier with a consistent set of features per example.

We can take an element-wise average of the word embeddings of the words contained in each training and test example. This makes feature vector representations that can be used as training and test features for our classifier. 

In [28]:
X_train_feature_vector = []
for w in X_train_word_embeddings:
    if w.size:
        X_train_feature_vector.append(w.mean(axis=0))
    else:
        X_train_feature_vector.append(np.zeros(100, dtype=float))
        
X_test_feature_vector = []
for w in X_test_word_embeddings:
    if w.size:
        X_test_feature_vector.append(w.mean(axis=0))
    else:
        X_test_feature_vector.append(np.zeros(100, dtype=float))

Each example now consists of one feature, which is a numerical feature vector of length 100. Run the code cell below to inspect the first five training examples.

In [29]:
for w in range(0, 5):
    print('Length of training example {0}: {1}'.format(w, len(X_train_feature_vector[w])))
    
print('First training example\'s feature vector: \n{0}'.format(X_train_feature_vector[0]))

Length of training example 0: 100
Length of training example 1: 100
Length of training example 2: 100
Length of training example 3: 100
Length of training example 4: 100
First training example's feature vector: 
[-1.3720958   0.12501732 -0.6105341   0.17654924 -0.28300652 -0.525982
  0.32349402  1.0935074   0.01302695 -0.67338824 -0.17330575 -0.50765145
 -0.5330617  -0.5382034  -0.35186034  0.10056937  0.01053756  0.1965194
  0.69199    -0.73445594  1.9991535   0.7262506   0.15985884 -0.27828264
 -0.02942562  0.5526415  -0.97427624 -0.48437983 -0.88097507 -0.07837696
 -1.1465707   1.3547388   0.38522092 -0.53465176  1.5360937   0.9801849
 -0.44139656  0.47553056  0.17276691 -0.9734109   0.3425239   0.26462913
 -1.273916    0.42164546  1.3260773  -1.7394804  -0.18339173 -0.52812415
  0.8996323   0.38009188  0.2649963  -0.22109905  0.38314363  0.5265235
  0.4519514   1.2999127   1.0125622   0.15200558 -1.0808791   0.18974045
 -0.22553766  0.5401875  -0.05285547 -0.61595297 -1.4598138   0

## Step 7: Fit a Logistic Regression Model to the Training Data and Evaluate the Model

Now we can train our model on our transformed data. The code cell below trains a logistic regression model and computes the AUC on the test set.

In [30]:
# 1. Create a LogisticRegression model object, and fit a Logistic Regression model to the transformed training data
model = LogisticRegression(max_iter=200)
model.fit(X_train_feature_vector, y_train)

# 2. Make predictions on the transformed test data using the predict_proba() method and 
# save the values of the second column
probability_predictions = model.predict_proba(X_test_feature_vector)[:,1]

# 3. Make predictions on the transformed test data using the predict() method 
class_label_predictions = model.predict(X_test_feature_vector)

# 4. Compute the Area Under the ROC curve (AUC) for the test data. Note that this time we are using one 
# function 'roc_auc_score()' to compute the auc rather than using both 'roc_curve()' and 'auc()' as we have 
# done in the past
auc = roc_auc_score(y_test, probability_predictions)
print('AUC on the test data: {:.4f}'.format(auc))

AUC on the test data: 0.9873


Let's check two emails and see if our model properly predicted whether an email is spam or not spam.

In [31]:
print('Email #1:\n')
print(original_X[14])

print('\nPrediction: Is this a spam email? {}\n'.format(class_label_predictions[14])) 

print('Actual: Is this a spam email? {}\n'.format(y_test.to_numpy()[14]))


Email #1:

Subject: tenaska iv july
darren :
please remove the price on the tenaska iv sale , deal 384258 , for july and enter the demand fee . the amount should be $ 3 , 902 , 687 . 50 .
thanks ,
megan

Prediction: Is this a spam email? True

Actual: Is this a spam email? True



In [32]:
print('Email #2:\n')
print(original_X[132])

print('\nPrediction: Is this a spam email? {}\n'.format(class_label_predictions[132])) 

print('Actual: Is this a good spam email? {}\n'.format(y_test.to_numpy()[132]))

Email #2:

Subject: re : noms / actual flow for 3 / 29 / 01
we agree with the nomination .
" eileen ponton " on 03 / 30 / 2001 10 : 05 : 40 am
to : david avila / lsp / enserch / us @ tu , charlie stone / texas utilities @ tu , melissa
jones / texas utilities @ tu , hpl . scheduling @ enron . com ,
liz . bellamy @ enron . com
cc :
subject : noms / actual flow for 3 / 29 / 01
nom mcf mmbtu
24 , 583 24 , 999 25 , 674
btu = 1 . 027

Prediction: Is this a spam email? False

Actual: Is this a good spam email? False

